In [1]:
from dotenv import load_dotenv
import os
load_dotenv()
neon_url = os.getenv("NEONDB_DATABASE_URL")
render_url = os.getenv("RENDER_DATABASE_URL")
xata_url = os.getenv("XATA_DATABASE_URL")

In [6]:
import psycopg
import time
import statistics
import os

# Avg Connection Latency of NeonDB: 494.19 ms

In [7]:
conn_str = os.getenv("NEONDB_DATABASE_URL")
latencies = []

for _ in range(100):
    start = time.perf_counter()
    conn = psycopg.connect(conn_str)
    conn.close()
    end = time.perf_counter()
    latencies.append((end - start) * 1000)  # ms

avg_latency = statistics.mean(latencies)
print(f"Avg Connection Latency of NeonDB: {avg_latency:.2f} ms")

Avg Connection Latency of NeonDB: 494.19 ms


# Avg Connection Latency of Render: 948.76 ms

In [8]:
conn_str = os.getenv("RENDER_DATABASE_URL")
latencies = []

for _ in range(100):
    start = time.perf_counter()
    conn = psycopg.connect(conn_str)
    conn.close()
    end = time.perf_counter()
    latencies.append((end - start) * 1000)  # ms

avg_latency = statistics.mean(latencies)
print(f"Avg Connection Latency of Render: {avg_latency:.2f} ms")

Avg Connection Latency of Render: 948.76 ms


# Avg Connection Latency of Xata: 984.92 ms

In [11]:
conn_str = os.getenv("XATA_DATABASE_URL")
latencies = []

for _ in range(100):
    start = time.perf_counter()
    conn = psycopg.connect(conn_str)
    conn.close()
    end = time.perf_counter()
    latencies.append((end - start) * 1000)  # ms

avg_latency = statistics.mean(latencies)
print(f"Avg Connection Latency of Xata: {avg_latency:.2f} ms")

Avg Connection Latency of Xata: 984.92 ms
